In [1]:
from nba_api.stats.endpoints import playercareerstats, playergamelog, playergamelogs
from nba_api.stats.static import players
import pandas as pd
from io import BytesIO

In [3]:
def get_player_id_from_name(player_name):
    all_players = players.get_players()
    for player in all_players:
        if player['full_name'].lower() == player_name.lower():
            return player['id']
    return None

In [15]:
from nba_api.stats.endpoints import playerprofilev2

player_name = "LeBron James"
player_id = get_player_id_from_name(player_name)
playerprofilev2.PlayerProfileV2(player_id).get_data_frames()[0]


,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,2544,2003-04,00,1610612739,CLE,19.0,79,79,3120,622,...,0.754,99,333,432,465,130,58,273,149,1654
1,2544,2004-05,00,1610612739,CLE,20.0,80,80,3388,795,...,0.750,111,477,588,577,177,52,262,146,2175
2,2544,2005-06,00,1610612739,CLE,21.0,79,79,3361,875,...,0.738,75,481,556,521,123,66,260,181,2478
3,2544,2006-07,00,1610612739,CLE,22.0,78,78,3190,772,...,0.698,83,443,526,470,125,55,250,171,2132
4,2544,2007-08,00,1610612739,CLE,23.0,75,74,3027,794,...,0.712,133,459,592,539,138,81,255,165,2250
5,2544,2008-09,00,1610612739,CLE,24.0,81,81,3054,789,...,0.780,106,507,613,587,137,93,241,139,2304
6,2544,2009-10,00,1610612739,CLE,25.0,76,76,2966,768,...,0.767,71,483,554,651,125,77,261,119,2258
7,2544,2010-11,00,1610612748,MIA,26.0,79,79,3063,758,...,0.759,80,510,590,554,124,50,284,163,2111
8,2544,2011-12,00,1610612748,MIA,27.0,62,62,2326,621,...,0.771,94,398,492,387,115,50,213,96,1683
9,2544,2012-13,00,1610612748,MIA,28.0,76,76,2877,765,...,0.753,97,513,610,551,129,67,226,110,2036


In [18]:
# Set player name and retrieve player ID
player_name = "LeBron James"
player_id = get_player_id_from_name(player_name)

# Retrieve player career statistics
career = playercareerstats.PlayerCareerStats(player_id=player_id)
career_stats = career.get_data_frames()[0]
career_stats = career_stats.rename(columns={'TEAM_ABBREVIATION': 'TEAM', 'PLAYER_AGE': 'AGE'})

# Define column lists
totals_cols = ['MIN', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'FGM', 'FGA',
            'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
            'OREB', 'DREB', 'PF']
per_game_cols = ['MPG', 'PPG', 'RPG', 'APG', 'SPG', 'BPG', 'TPG', 'FGM', 'FGA',
                'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
                'ORPG', 'DRPG', 'PF']
percentage_stats = ['FG_PCT', 'FG3_PCT', 'FT_PCT']

# Career Totals
career_totals_per_season = career_stats[['SEASON_ID', 'AGE', 'GP', 'TEAM'] + totals_cols].copy()
career_totals_per_season['FULL_NAME'] = player_name

# Sum career totals and convert to int
career_totals = {'FULL_NAME': player_name, 'GP': int(career_totals_per_season['GP'].sum())}
career_totals.update(career_totals_per_season[totals_cols].sum().astype(int).to_dict())

# Convert non-integer columns to int except for 'FULL_NAME', 'SEASON_ID', and percentage columns
for col in career_totals_per_season.columns:
    if col not in ['FULL_NAME', 'SEASON_ID', 'TEAM'] + percentage_stats:
        career_totals_per_season[col] = career_totals_per_season[col].astype(int)

# Career Averages
career_avg_per_season = pd.DataFrame(columns=['FULL_NAME', 'SEASON_ID', 'AGE', 'TEAM', 'GP'] + per_game_cols)

# Calculate per game averages
for stat, per_game_stat in zip(totals_cols, per_game_cols):
    career_avg_per_season[per_game_stat] = round(career_stats[stat] / career_stats['GP'], 1)

career_avg_per_season['FULL_NAME'] = player_name
career_avg_per_season['SEASON_ID'] = career_stats['SEASON_ID']
career_avg_per_season['AGE'] = career_stats['AGE']
career_avg_per_season['TEAM'] = career_stats['TEAM']
career_avg_per_season['GP'] = career_stats['GP']
career_avg_per_season['AGE'] = career_avg_per_season['AGE'].astype('int')

# Calculate career averages
career_averages = {'FULL_NAME': player_name, 'GP': int(career_avg_per_season['GP'].sum())}
career_averages.update(round(career_avg_per_season[per_game_cols].mean(), 1).to_dict())

# Convert percentage Stats
for stat in percentage_stats:
    career_avg_per_season[stat] = round(career_stats[stat] * 100, 1)
    career_averages[stat] = round(career_avg_per_season[stat].mean(), 1)
    career_totals_per_season[stat] = round(career_totals_per_season[stat] * 100, 1)
    career_totals[stat] = round(career_avg_per_season[stat].mean(), 1)

# Convert DataFrames to dictionaries
career_totals_per_season_json = career_totals_per_season.to_dict(orient='records')
career_avg_per_season_json = career_avg_per_season.to_dict(orient='records')

career_totals_per_season

,SEASON_ID,AGE,GP,TEAM,MIN,PTS,REB,AST,STL,BLK,...,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,PF,FULL_NAME
0,2003-04,19,79,CLE,3120,1654,432,465,130,58,...,63,217,29.0,347,460,75.4,99,333,149,LeBron James
1,2004-05,20,80,CLE,3388,2175,588,577,177,52,...,108,308,35.1,477,636,75.0,111,477,146,LeBron James
2,2005-06,21,79,CLE,3361,2478,556,521,123,66,...,127,379,33.5,601,814,73.8,75,481,181,LeBron James
3,2006-07,22,78,CLE,3190,2132,526,470,125,55,...,99,310,31.9,489,701,69.8,83,443,171,LeBron James
4,2007-08,23,75,CLE,3027,2250,592,539,138,81,...,113,359,31.5,549,771,71.2,133,459,165,LeBron James
5,2008-09,24,81,CLE,3054,2304,613,587,137,93,...,132,384,34.4,594,762,78.0,106,507,139,LeBron James
6,2009-10,25,76,CLE,2966,2258,554,651,125,77,...,129,387,33.3,593,773,76.7,71,483,119,LeBron James
7,2010-11,26,79,MIA,3063,2111,590,554,124,50,...,92,279,33.0,503,663,75.9,80,510,163,LeBron James
8,2011-12,27,62,MIA,2326,1683,492,387,115,50,...,54,149,36.2,387,502,77.1,94,398,96,LeBron James
9,2012-13,28,76,MIA,2877,2036,610,551,129,67,...,103,254,40.6,403,535,75.3,97,513,110,LeBron James


In [16]:
5.538095238095238/7.55238095238095

0.7332912988650695

In [56]:
def get_player_image(player_name):
    """
    Gets NBA player headshot using player ID.

    Args:
        - player_name (str): Full name of current or former NBA player.

    Returns:
        - URL link to the headshot of NBA player.
    """

    player_id = get_player_id_from_name(player_name=player_name)
    url = f'https://cdn.nba.com/headshots/nba/latest/1040x760/{player_id}.png'

    return url

get_player_image("Jason Kidd")

'https://cdn.nba.com/headshots/nba/latest/1040x760/467.png'